### **EQUIPE**

---

**POR FAVOR, PREENCHER OS INTEGRANDES DA SUA EQUIPE:**


**Integrante 01:**

`Por favor, informe o seu nome completo e RA:`

Thiago Perellon Romano - 11202130849


### **GRANDE MODELO DE LINGUAGEM (*Large Language Model - LLM*)**

---

Cada equipe deve selecionar um Grande Modelo de Linguagem (*Large Language Model - LLM*). Cada modelo pode ser escolhido por até 5 equipes.



Por favor, informe os dados do LLM selecionada:

>


**LLM**: Groq (llama-3.1)

>

**Link para a documentação oficial**: https://console.groq.com/docs/quickstart



### **API**
---

Por favor, informe os dados da API selecionada:

**API**: NewsAPI

**Site oficial**: https://newsapi.org/

**Link para a documentação oficial**: https://newsapi.org/docs






As seguintes técnicas de PLN foram usadas:

*   Classificação de Textos
*   Extração de Palavras-chave
*   Tradução de Textos
*   Sumarização de Textos



#Instalação dos pacotes necessários

In [19]:
!pip install langchain
!pip install langchain-groq
!pip install langchainhub
!pip install langchain_community
!pip install langchain-text-splitters
!pip install fastembed
!pip install tiktoken
!pip install firecrawl-py
!pip install faiss-cpu
!pip install transformers
!pip install yake

##Importando as bibliotecas usadas para fazer as requisições de API.

In [20]:
import requests
from google.colab import userdata

Como pode ser visto, a NewsAPI não retorna o conteúdo das notícias brasileiras

In [21]:
import json
# URL base da API
URL_BASE = 'https://newsapi.org/v2/top-headlines'
# Parâmetros da requisição
params = {
    'sources': 'google-news-br',
    'apiKey': userdata.get('NEWS_API_KEY'),
    'pageSize': 1  # Limitando a 1 resultado
}

# Fazendo a requisição GET
response = requests.get(URL_BASE, params=params)

# Verificando se a requisição foi bem-sucedida
if response.status_code == 200:
    data = response.json()
    print(json.dumps(data, indent=2, ensure_ascii=False))
else:
    print(f"Falha na requisição.")

{
  "status": "ok",
  "totalResults": 10,
  "articles": [
    {
      "source": {
        "id": "google-news-br",
        "name": "Google News (Brasil)"
      },
      "author": "SBT",
      "title": "Ônibus tomba no Rio de Janeiro e deixa 26 feridos - SBT",
      "description": null,
      "url": "https://news.google.com/rss/articles/CBMilwFBVV95cUxPSGhBdGFlSl9jSS1ZWGRqSElUcTVMRl9MZHhUdklyZnpDc1M1YnBlSzJsU1dqdGc5d041MXIxM3p1RDJlcTB3Nm1pUFd5M25sS1BBY0c2aXVQSXJCdUVZV3FUUUxuYkQ2SF90MGY2enMyMXhKb0hBc29ka2J2TXJ2N2JsbC1nQUZBMm9ISVdoRzJtLUlidzBJ?oc=5",
      "urlToImage": null,
      "publishedAt": "2024-09-08T00:37:00+00:00",
      "content": null
    }
  ]
}


E para notícias estrangeiras retorna o conteúdo truncado, ou seja, será necessário realizar web scraping nas urls que a API retorna.


In [22]:
import json
# URL base da API
URL_BASE = 'https://newsapi.org/v2/top-headlines'
# Parâmetros da requisição
params = {
    'country':'us',
    'apiKey': userdata.get('NEWS_API_KEY'),
    'pageSize': 1  # Limitando a 1 resultado
}

# Fazendo a requisição GET
response = requests.get(URL_BASE, params=params)

# Verificando se a requisição foi bem-sucedida
if response.status_code == 200:
    data = response.json()
    print(json.dumps(data, indent=2, ensure_ascii=False))
else:
    print(f"Falha na requisição.")

{
  "status": "ok",
  "totalResults": 37,
  "articles": [
    {
      "source": {
        "id": "cnn",
        "name": "CNN"
      },
      "author": "Raja Razek, Emma Tucker",
      "title": "Ketucky I-75 shooting: Numerous people shot in Kentucky near Interstate 75, officials say - CNN",
      "description": "Several people have been shot near Interstate 75 in Laurel County, Kentucky, according to the Laurel County Sheriff’s office.",
      "url": "https://www.cnn.com/2024/09/07/us/kentucky-interstate-75-shooting/index.html",
      "urlToImage": "https://media.cnn.com/api/v1/images/stellar/prod/screen-shot-2024-09-07-at-7-12-13-pm.jpg?c=16x9&q=w_800,c_fill",
      "publishedAt": "2024-09-08T00:36:00Z",
      "content": "A manhunt is underway for at least one suspect after several people were shot Saturday near an interstate in a rural area of Laurel County, Kentucky, located south of Lexington, according to the sher… [+2985 chars]"
    }
  ]
}


Adiantando um problema futuro: URLs das notícias brasileiras são links redirecionadores e o FireCrawl não consegue acessar e não consegui encontrar uma forma de obter a URL final. Portanto, usarei notícias de fora do país.

PARA O PROGRAMA RODAR MAIS RÁPIDO, TROQUE 'pageSize' PARA UM VALOR MENOR

In [23]:
# Faz requisições as APIs e retorna uma lista de URLs das notícias encontradas
def fetch_news():
  links = []
  URL_BASE = 'https://newsapi.org/v2/top-headlines'
  params = {
      'country': 'us', # Define o país, Brasil não retorna informações válidas
      'apiKey': userdata.get('NEWS_API_KEY'), # key da api
      'pageSize': 5, # número de resultados
  }
  response = requests.get(URL_BASE, params=params)

  # Verifica se a requisição foi bem-sucedida
  if response.status_code == 200:
    data = response.json()
    for article in data['articles']:
      links.append(article.get('url'))
    return links
  else:
    return None

links_teste = fetch_news()
print(links_teste)

['https://www.cnn.com/2024/09/07/us/kentucky-interstate-75-shooting/index.html', 'https://sports.yahoo.com/live/college-football-week-2-scores-games-colorado-vs-nebraska-tennessee-vs-nc-state-and-more-150045723.html', 'https://www.washingtonpost.com/investigations/2024/09/07/apalachee-georgia-oxford-shooting/', 'https://www.washingtonpost.com/world/2024/09/07/israel-gaza-hamas-west-bank-aysenur-eygi-investigation/', 'https://www.npr.org/2024/09/07/nx-s1-5104786/tennis-us-open-aryna-sabalenka-beats-jessica-pegula']


A função crawl_e_save() é responsável por utilizar o FireCrawlLoader para fazer scraping das URLs e extrair as informações. Depois divide o conteúdo dos documentos em partes menores, salva em um arquivo TXT único para cada notícia, trata possíveis erros no processo de scraping e salvamento.

In [24]:
from langchain_community.document_loaders import FireCrawlLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

def crawl_e_salva(urls):
    for index, url in enumerate(urls, start=1):
        try:
            # Inicializar o FireCrawlLoader com a URL
            loader = FireCrawlLoader(
                api_key=userdata.get('FIRECRAWL_API_KEY'),
                url=url,
                mode='scrape')

            # Carrega os documentos
            documents = loader.load()

            # Cria um splitter de texto
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=2500, chunk_overlap=200)

            # Divide os documentos em chunks menores
            splits = text_splitter.split_documents(documents)

            # Define o nome dos arquivos
            filename = f"noticia_{index}.txt"

            # Salva o conteúdo em um arquivo .txt
            with open(filename, "w", encoding="utf-8") as f:
                f.write(f"URL original: {url}\n\n")  # Adiciona a URL original no início do arquivo
                for split in splits:
                    f.write(split.page_content + "\n\n")

            print(f"Conteúdo de {url} salvo em {filename}")

        except Exception as e:
            print(f"Erro ao processar {url}: {str(e)}")

# Lista de URLs para crawlear
urls_to_crawl = fetch_news()
crawl_e_salva(urls_to_crawl)

Conteúdo de https://www.cnn.com/2024/09/07/us/kentucky-interstate-75-shooting/index.html salvo em noticia_1.txt
Conteúdo de https://sports.yahoo.com/live/college-football-week-2-scores-games-colorado-vs-nebraska-tennessee-vs-nc-state-and-more-150045723.html salvo em noticia_2.txt
Conteúdo de https://www.washingtonpost.com/investigations/2024/09/07/apalachee-georgia-oxford-shooting/ salvo em noticia_3.txt
Conteúdo de https://www.washingtonpost.com/world/2024/09/07/israel-gaza-hamas-west-bank-aysenur-eygi-investigation/ salvo em noticia_4.txt
Conteúdo de https://www.npr.org/2024/09/07/nx-s1-5104786/tennis-us-open-aryna-sabalenka-beats-jessica-pegula salvo em noticia_5.txt


In [25]:
# Importando o restante dos pacotes instalados no começo
import os
from typing import List
from pydantic import BaseModel, Field
from langchain.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_text_splitters import TokenTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_core.runnables import RunnableLambda
import textwrap
from IPython.display import display, Markdown
from transformers import pipeline
import yake

# Definição da classe Noticia usando Pydantic para validação de dados

In [26]:
class Noticia(BaseModel):
    titulo: str = Field(..., description="Título da notícia")
    url: str = Field(..., description="URL original da notícia")
    fonte: str = Field(..., description="Fonte da notícia")
    resumo: str = Field(..., description="Um resumo detalhado da notícia")
    categoria: str = Field(None, description="Categoria da notícia")
    palavras_chave: List[str] = Field(default_factory=list, description="Palavras-chave da notícia")

# Configuração do prompt

In [37]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """Você é um especialista em análise de notícias. Sua tarefa é extrair
    as informações mais importantes do texto fornecido. É CRUCIAL que você forneça
    APENAS as seguintes informações, exatamente neste formato:

    Título: [Título preciso da notícia]
    Fonte: [Nome do jornal ou site]
    Resumo: [Um resumo conciso da notícia em 3-5 frases, totalizando 75-100 palavras]

    O resumo deve incluir os principais fatos, pessoas ou organizações envolvidas,
    contexto relevante e impacto ou importância da notícia. Seja objetivo e
    informativo, evitando opiniões ou especulações."""),
    ("human", "{texto}"),
])

# Configura o modelo de chat usando o ChatGroq com o modelo llama-3.1 da META/Facebook

In [28]:
modelo = ChatGroq(
    model="llama-3.1-70b-versatile",
    api_key=userdata.get('GROQ_API_KEY'),
    temperature=0.1
)

extrator = prompt | modelo

# Configuração do splitter de texto

In [29]:
text_splitter = TokenTextSplitter(chunk_size=2000, chunk_overlap=100)

# Configuração do armazenamento vetorial

In [30]:
embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-large-en-v1.5")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

#Função de classificação de texto

In [31]:
# Inicializar o pipeline de classificação de texto
classificador = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Função para classificar texto
def classificar_noticia(texto: str, categorias: List[str]) -> str:
    resultado = classificador(texto, candidate_labels=categorias)
    return resultado['labels'][0]  # Retorna a categoria com maior pontuação

kw_extractor = yake.KeywordExtractor(lan="pt", n=1, top=5)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


#Função de extração de palavras-chave

In [32]:
# Função para extrair palavras-chave
def extrair_palavras_chave(texto: str, top_n: int = 5) -> List[str]:
    palavras_chave = kw_extractor.extract_keywords(texto)

    # Retorna as palavras-chave como uma lista de strings
    return [palavra[0] for palavra in palavras_chave[:top_n]]

#Processar arquivos

Aqui é onde as notícias são processadas e são resumidas, categorizadas e tem suas palavras-chave extraídas.

In [33]:
def processar_arquivo(arquivo: str) -> Noticia:
    with open(arquivo, 'r', encoding='utf-8') as f:
        conteudo = f.read()

    # Extrair a URL original do conteúdo
    url = conteudo.split('\n')[0].replace("URL original: ", "").strip()

    # Divide o conteúdo em partes menores
    textos = text_splitter.split_text(conteudo)
    vectorstore = FAISS.from_texts(textos, embedding=embeddings)
    # Busca os 3 documentos mais relevantes
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

    # Função paara formatar documentos
    def documentos(docs):
        return "\n".join([doc.page_content for doc in docs])

    # Configura o extrator de informações usando RAG
    rag_extrator = {
        "texto": retriever | RunnableLambda(documentos)
    } | extrator

    resultado = rag_extrator.invoke("Extraia as informações principais desta notícia")

    # Processar o resultado para extrair as informações
    linhas = resultado.content.split('\n')
    titulo = "Título não encontrado"
    fonte = "Fonte não identificada"
    resumo = ""

    for linha in linhas:
        if linha.startswith("Título:"):
            titulo = linha.split(":", 1)[1].strip()
        elif linha.startswith("Fonte:"):
            fonte = linha.split(":", 1)[1].strip()
        elif linha.startswith("Resumo:"):
            resumo = linha.split(":", 1)[1].strip()

    # Classificar o resumo da notícia em categorias
    categorias = ["Política", "Economia", "Esportes", "Tecnologia", "Saúde", "Entretenimento", "Crimes"]
    categoria = classificar_noticia(resumo, categorias)

    # Extrair palavras-chave do resumo da notícia
    palavras_chave = extrair_palavras_chave(resumo)

    # Retorna um objeto Noticia com as informações extraídas
    return Noticia(
        titulo=titulo,
        url=url,
        fonte=fonte,
        resumo=resumo,
        categoria=categoria,
        palavras_chave=palavras_chave
    )

Itera a função acima para cada arquivo no diretório atual que corresponda ao formato do nome

In [34]:
diretorio = '.'  # Diretório atual, onde os arquivos .txt são salvos
arquivos = [f for f in os.listdir(diretorio) if f.startswith('noticia_') and f.endswith('.txt')]

todas_noticias = []

for arquivo in sorted(arquivos):
    print(f"Processando {arquivo}...")
    noticia = processar_arquivo(os.path.join(diretorio, arquivo))
    todas_noticias.append(noticia)

Processando noticia_1.txt...
Processando noticia_2.txt...
Processando noticia_3.txt...
Processando noticia_4.txt...
Processando noticia_5.txt...


#Salva todas as notícias em um arquivo Markdown

Para ficar melhor formatado.

In [35]:
# Salvar todas as notícias em um único arquivo Markdown para ficar melhor formatado
with open('resumo_noticias.md', 'w', encoding='utf-8') as f:
    f.write("# Resumo de Todas as Notícias\n\n")
    for noticia in todas_noticias:
        f.write(f"## {noticia.titulo}\n\n")
        f.write(f"**URL:** {noticia.url}\n\n")
        f.write(f"**Fonte:** {noticia.fonte}\n\n")
        f.write(f"**Categoria:** {noticia.categoria}\n\n")
        f.write("### Palavras-chave\n\n")
        f.write(", ".join(noticia.palavras_chave) + "\n\n")
        f.write("### Resumo\n\n")
        resumo_formatado = textwrap.fill(noticia.resumo, width=80)
        f.write(f"{resumo_formatado}\n\n---\n\n")

Imprime o conteúdo final do programa.

In [36]:
def exibir_md(arquivo: str):
    # Ler o conteúdo do arquivo Markdown
    with open(arquivo, 'r', encoding='utf-8') as f:
        conteudo_md = f.read()

    # Exibir o conteúdo do Markdown no Google Colab
    display(Markdown(conteudo_md))

exibir_md("resumo_noticias.md")

# Resumo de Todas as Notícias

## Manhunt underway after 5 people injured in Kentucky shooting near Interstate 75, officials say

**URL:** https://www.cnn.com/2024/09/07/us/kentucky-interstate-75-shooting/index.html

**Fonte:** CNN

**Categoria:** Crimes

### Palavras-chave

Interstate, autoridades, Kentucky, caçada, humana

### Resumo

Uma caçada humana está em andamento após cinco pessoas terem sido feridas em um
tiroteio perto da Interstate 75, no Kentucky. As autoridades estão procurando
por um suspeito identificado como Joseph A. Couch, de 32 anos, que é considerado
"armado e perigoso". O tiroteio ocorreu em uma área rural do condado de Laurel,
cerca de 80 milhas ao sul de Lexington. As vítimas estão em estado estável e as
autoridades estão pedindo que as pessoas se mantenham vigilantes e evitem a
área. O governador do Kentucky, Andy Beshear, disse que as autoridades estão
"ativamente monitorando a situação" e oferecendo apoio.

---

## Resultados da Semana 2 do Futebol Americano Universitário: Oregon vence Boise State com um field goal no último segundo

**URL:** https://sports.yahoo.com/live/college-football-week-2-scores-games-colorado-vs-nebraska-tennessee-vs-nc-state-and-more-150045723.html

**Fonte:** Yahoo Sports

**Categoria:** Esportes

### Palavras-chave

Semana, State, Futebol, Americano, Universitário

### Resumo

A Semana 2 do Futebol Americano Universitário foi marcada por várias partidas
emocionantes. Oregon venceu Boise State com um field goal no último segundo,
enquanto Texas derrotou Michigan por 31-12. Além disso, Northern Illinois
surpreendeu Notre Dame com uma vitória por 16-14, e USC venceu Utah State por
48-0. Outras partidas notáveis incluíram a vitória de Kansas State sobre Tulane
e a vitória de Oklahoma State sobre Arkansas em duplo tempo extra.

---

## Pai de atirador de escola é julgado após filho de 14 anos matar quatro pessoas em escola na Geórgia

**URL:** https://www.washingtonpost.com/investigations/2024/09/07/apalachee-georgia-oxford-shooting/

**Fonte:** The Washington Post

**Categoria:** Crimes

### Palavras-chave

Geórgia, escola, Gray, matou, julgado

### Resumo

O pai de um atirador de escola de 14 anos que matou quatro pessoas em uma escola
na Geórgia está sendo julgado. Colin Gray, 54 anos, é o pai de Colt Gray, que
foi acusado de quatro assassinatos. A promotora Karen McDonald havia debatido se
deveria processar os pais do atirador de Oxford High School, que matou quatro
estudantes em 2021. A decisão de processar os pais é inédita e pode estabelecer
um precedente.

---

## Autoridades palestinas investigam morte de cidadã americana em protesto na Cisjordânia

**URL:** https://www.washingtonpost.com/world/2024/09/07/israel-gaza-hamas-west-bank-aysenur-eygi-investigation/

**Fonte:** The Washington Post

**Categoria:** Crimes

### Palavras-chave

Cisjordânia, Beita, Aysenur, Eygi, cidadã

### Resumo

Aysenur Eygi, uma cidadã americana de 26 anos, foi morta em um protesto na
Cisjordânia, na cidade de Beita, após ser atingida por um tiro na cabeça
disparado por forças israelenses. As autoridades palestinas começaram a coletar
depoimentos de testemunhas para investigar o incidente. Eygi era uma ativista
voluntária que participava do protesto. A morte dela é o mais recente episódio
de violência na região, que tem sido marcada por conflitos entre israelenses e
palestinos.

---

## Aryna Sabalenka vence Jessica Pegula e conquista seu primeiro US Open

**URL:** https://www.npr.org/2024/09/07/nx-s1-5104786/tennis-us-open-aryna-sabalenka-beats-jessica-pegula

**Fonte:** NPR

**Categoria:** Entretenimento

### Palavras-chave

Sabalenka, Bielorrússia, EUA, Open, Jessica

### Resumo

Aryna Sabalenka, da Bielorrússia, conquistou seu primeiro título do US Open
feminino ao derrotar Jessica Pegula, dos EUA, por 7-5, 7-5. Essa é a terceira
vitória de Sabalenka em torneios Grand Slam. A vitória ocorreu no sábado, em
Nova York, e marca um importante momento na carreira de Sabalenka, que havia
sido vice-campeã no ano anterior. Pegula, que estava disputando sua primeira
final de Grand Slam, lutou até o fim, mas não conseguiu superar a habilidade de
Sabalenka.

---

